In [1]:
!pip install tensorflow opencv-python numpy matplotlib


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt


In [3]:
!unzip -q "/content/dataset.zip" -d "/content/"



In [4]:
# Define paths
train_dir = "/content/dataset/train"  # Your training folder
test_dir = "/content/dataset/test"    # Your testing folder

# Define image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32

# Data Augmentation and Preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_data = datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'  # Binary classification
)

test_data = datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)


Found 28707 images belonging to 2 classes.
Found 7178 images belonging to 2 classes.


In [5]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, LeakyReLU

# Load VGG16 without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Unfreeze last 4 layers for fine-tuning
for layer in base_model.layers[:-4]:  # Freeze all layers except last 4
    layer.trainable = False

# Custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Convert feature maps to a single vector

# First Dense Layer with Batch Norm and LeakyReLU
x = Dense(256)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(0.4)(x)

# Second Dense Layer
x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(0.3)(x)

# Output Layer (Binary Classification)
output_layer = Dense(1, activation='sigmoid')(x)  # Binary classification (stress vs non-stress)

# Create the model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model with AdamW optimizer
model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=0.0001, weight_decay=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Model Summary
model.summary()


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         131,3

 Total params: 14,880,577 (56.76 MB)

 Trainable params: 7,244,545 (27.64 MB)

 Non-trainable params: 7,636,032 (29.13 MB)

In [6]:
# Early Stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Reduce Learning Rate on Plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Train the model
EPOCHS = 20

history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=EPOCHS,
    callbacks=[early_stopping, reduce_lr]
)

# Save the trained model
model.save("stress_detector.h5")


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 478s 513ms/step - accuracy: 0.5854 - loss: 0.7344 - val_accuracy: 0.6839 - val_loss: 0.5890 - learning_rate: 1.0000e-04
Epoch 2/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 450s 501ms/step - accuracy: 0.6792 - loss: 0.6015 - val_accuracy: 0.7300 - val_loss: 0.5287 - learning_rate: 1.0000e-04
Epoch 3/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 450s 502ms/step - accuracy: 0.7210 - loss: 0.5480 - val_accuracy: 0.6530 - val_loss: 0.7169 - learning_rate: 1.0000e-04
Epoch 4/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 502s 502ms/step - accuracy: 0.7402 - loss: 0.5208 - val_accuracy: 0.6644 - val_loss: 0.5945 - learning_rate: 1.0000e-04
Epoch 5/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 449s 500ms/step - accuracy: 0.7530 - loss: 0.4952 - val_accuracy: 0.7601 - val_loss: 0.4894 - learning_rate: 1.0000e-04
Epoch 6/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 449s 500ms/step - accuracy: 0.7666 - loss: 0.4787 - val_accuracy: 0.7760 - val_loss: 0.4610 - learning_rate: 1.0000e-04
Epoch 7/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 

In [7]:
# Evaluate on test data
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


225/225 ━━━━━━━━━━━━━━━━━━━━ 88s 390ms/step - accuracy: 0.8464 - loss: 0.3458
Test Accuracy: 81.36%


In [8]:
def classify_stress_level(probability):
    """Categorize stress level based on the probability of stress."""
    if probability < 0.5:
        return "Non-Stress"
    elif 0.5 <= probability < 0.65:
        return "Low Stress"
    elif 0.65 <= probability < 0.85:
        return "Medium Stress"
    else:
        return "High Stress"


In [ ]:
# image-based stress detection

# Load trained model
model = tf.keras.models.load_model("stress_detector.h5")

# Function to predict stress level from an image
def predict_stress(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    # Predict probability of stress
    probability = model.predict(img)[0][0]  # Extract single probability value
    stress_category = classify_stress_level(probability)

    # Display result
    plt.imshow(cv2.imread(image_path))
    plt.title(f"Stress Level: {stress_category} ({probability:.2f})")
    plt.show()

    return stress_category, probability

# Example usage
stress_label, prob = predict_stress("test/some_image.jpg")
print(f"Predicted Stress Level: {stress_label} with probability {prob:.2f}")


In [ ]:
#Real-time webcam stress detection

# Open webcam and predict stress level in real-time
cap = cv2.VideoCapture(0)  # 0 for default webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to RGB and resize
    img = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)

    # Predict stress level
    probability = model.predict(img)[0][0]
    stress_label = classify_stress_level(probability)

    # Display result
    cv2.putText(frame, f"Stress Level: {stress_label}", (20, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Real-Time Stress Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
exercise_dict = {
    "Non-Stress": "Keep up the good work! Maintain a balanced routine.",
    "Low Stress": "Try light stretching, a short walk, or mindfulness breathing.",
    "Medium Stress": "Consider deep breathing exercises, yoga, or meditation.",
    "High Stress": "Engage in intense workouts, journaling, or professional counseling."
}

def recommend_exercise(stress_level):
    return exercise_dict.get(stress_level, "No recommendation available.")

# Example
print(f"Recommended Exercise: {recommend_exercise(stress_label)}")
